# Results

In [1]:
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
def get_runs(entity, project, log_keys=None, filters=None):
    api = wandb.Api(timeout=20)
    runs = api.runs(f"jccamml/{project}", filters=filters)
    print(f"Found {len(runs)} runs.")
    history = []
    for run in runs:
        if log_keys:
            for log in run.scan_history(keys=log_keys):
                log["config"] = dict(run.config)
                history.append(log)
        else:
            history.append({"config": dict(run.config), **dict(run.summary)})

    df = pd.json_normalize(history)
    return runs, df